# Data Processing
Collection of functions for cleaning up raw pressure readings.

## Contents
* [Data Retrieval](#retrieval)
* [Normalization](#normalization)
* [Interpolation](#interpolaton)
* [Masking](#masking)
* [Fourier Transform](#fft)
* [Constrained Inverse Fourier Transform](#ifft)
<hr>

### <a id='retrieval'> Data retrieval </a>

In [6]:
def retrieve_pressure_data(gauge_id, fillNumber, length=3000, fill=None):
    
    fill = db.getLHCFillData(fillNumber) if fill is None else fill
    t1, t2 = fill['startTime'], fill['endTime'] #+ 10000
    
    VG = db.get(gauge_id, t1, t2) # Returns a dictionary
    data = pd.DataFrame.from_dict(VG,dtype='float',columns=['Time Stamp','pressure'],orient='index')
    df = pd.DataFrame()
    pressure_readings = data["pressure"][0]
    time_readings = data["Time Stamp"][0]
    
    if isinstance(time_readings, (list, tuple, np.ndarray)) and np.sum(time_readings) != 0:
        t = np.linspace(time_readings[0], time_readings[-1], length)
        pressure_readings = np.interp(t, time_readings, pressure_readings)
        time_readings = t
    
    return time_readings, pressure_readings



In [9]:
def retrieve_beam_energy(fillNumber=0, length=3000, fill = None, t1=0, t2=0):
    if t1 == 0:
        fill = db.getLHCFillData(fillNumber) if fill is None else fill
        t1, t2 = fill['startTime'], fill['endTime'] #+ 10000
    
    energy_variable = 'LHC.BOFSU:OFSU_ENERGY'
    beamEnergy=db.get(energy_variable, t1, t2)
    beam_time, beam_energy = beamEnergy[energy_variable]
    
    if t1 == 0:
        t = np.linspace(beam_time[0], beam_time[-1], length)
        beam_energy = np.interp(t, beam_time, beam_energy)
    else:
        t = beam_time
    
    return t, beam_energy


In [3]:
def retrieve_beam_intensity(fillNumber=0, length=3000, fill = None, beamNumber=1, t1=0, t2=0):
    if t1 == 0:
        fill = db.getLHCFillData(fillNumber) if fill is None else fill
        t1, t2 = fill['startTime'], fill['endTime'] #+ 10000
    
    intensity_variable = 'LHC.BCTFR.A6R4.B'+str(beamNumber)+':BEAM_INTENSITY'
    beamIntensity=db.get(intensity_variable, t1, t2)
    intensity_time, beam_intensity = beamIntensity[intensity_variable]
    
    if t1 == 0:
        t = np.linspace(intensity_time[0], intensity_time[-1], length)
        beam_intensity = np.interp(t, intensity_time, beam_intensity)
    else:
        t = intensity_time    
    
    return t, beam_intensity

In [ ]:
def retrieve_beam_bunch_length(fillNumber=0, length=3000, fill = None, beamNumber=1, t1=0, t2=0):
    if t1 == 0:
        fill = db.getLHCFillData(fillNumber) if fill is None else fill
        t1, t2 = fill['startTime'], fill['endTime'] #+ 10000
    
    bunch_length_variable = 'LHC.BQM.B'+str(beamNumber)+':BUNCH_LENGTH_MEAN'
    buncLength=db.get(bunch_length_variable, t1, t2)
    bunch_length_time, bunch_length = buncLength[bunch_length_variable]
    
    if t1 == 0:
        t = np.linspace(bunch_length_time[0], bunch_length_time[-1], length)
        bunch_length = np.interp(t, bunch_length_time, bunch_length)
    else:
        t = bunch_length_time
    
    return t, bunch_length

In [2]:
def process_data_for_PCA(probe_ID, fill, verboose=False):
    
    time_readings, pressure_readings = retrieve_gauge_data(probe_ID,fill)
    
    t = np.linspace(time_readings[0], time_readings[-1], length)

    
    return time_readings, pressure_readings

### <a id='normalization'> Normalization </a>

In [10]:
def normalize_y(time_readings, beam_time, pressure_readings, beam_energy, show_plot=False):
    if  np.max(pressure_readings) - np.min(pressure_readings) != 1: #is not normalized already
        norm_pressure_readings = ( pressure_readings - np.min(pressure_readings) ) /  ( np.max(pressure_readings) - np.min(pressure_readings))

    if  np.max(beam_energy) - np.min(beam_energy) != 1: #is not normalized already
        norm_beam_energy = ( beam_energy - np.min(beam_energy) ) /  ( np.max(beam_energy) - np.min(beam_energy))
        
    if show_plot:
        f, ax = plt.subplots(1,figsize=(7,5))
        advancedPlottingUtility(axes=[ax],axis_index=0,
                                xs=[time_readings,beam_time],ys=[norm_pressure_readings,norm_beam_energy],
                                time_x=True,
                                plot_labels=("Vacuum Gauge","Beam Energy"),
                                axis_labels=("Time Stamp","Norm. Pressure","Norm. Energy"),
                                title="Data Normalizaton")
    return norm_pressure_readings,norm_beam_energy


### <a id='interpolaton'> Interpolation </a>

In [11]:
def interpolate_readings(x, y, length = 3000, verbose=False):
    # Vacuum Gauge
    
    t = np.linspace(time_readings[0], time_readings[-1], length)
    if verbose:
        print("t {}, time_readings {}, pressure_readings {}".format(t.shape,time_readings.shape,pressure_readings.shape))
        
    pressure_readings = np.interp(t, time_readings, pressure_readings)
    # Beam Intensity
    t2 = np.linspace(beam_time[0], beam_time[-1], density_fact*len(time_readings))
    beam_energy = np.interp(t2, beam_time, beam_energy)

    time_readings = np.linspace(0,1,len(t))
    beam_time = np.linspace(0,1,len(t2))
    
    return time_readings,pressure_readings,beam_time,beam_energy

